# 1. Đọc dữ liệu

In [6]:
import pandas as pd
import numpy as np
import os

# --- 1. LOAD DỮ LIỆU ---
print("Đang đọc dữ liệu từ file CSV...")
path = './' 

orders = pd.read_csv(path + 'olist_orders_dataset.csv')
items = pd.read_csv(path + 'olist_order_items_dataset.csv')
products = pd.read_csv(path + 'olist_products_dataset.csv')
payments = pd.read_csv(path + 'olist_order_payments_dataset.csv')
reviews = pd.read_csv(path + 'olist_order_reviews_dataset.csv')
customers = pd.read_csv(path + 'olist_customers_dataset.csv')
sellers = pd.read_csv(path + 'olist_sellers_dataset.csv')
translation = pd.read_csv(path + 'product_category_name_translation.csv')

print("Đã đọc xong! Bắt đầu xử lý...")


Đang đọc dữ liệu từ file CSV...
Đã đọc xong! Bắt đầu xử lý...


# 2. Dịch tên sản phẩm

In [7]:
# Nối bảng Product với bảng Translation để lấy tên tiếng Anh
products = pd.merge(products, translation, on='product_category_name', how='left')

# Chỉ giữ lại cột tên tiếng Anh, bỏ tên Bồ Đào Nha cho đỡ rối
products['product_category_name'] = products['product_category_name_english']
products.drop(columns=['product_category_name_english'], inplace=True)

# 3. Nối bảng

In [8]:
# Nguyên tắc: Lấy bảng 'orders' làm gốc, nối các bảng khác vào

# Bước 3.1: Nối chi tiết sản phẩm (Items)
# Lưu ý: Một đơn hàng có thể có nhiều sản phẩm -> Số dòng sẽ tăng lên
df = pd.merge(orders, items, on='order_id', how='left')

# Bước 3.2: Nối thông tin Sản phẩm (Products)
df = pd.merge(df, products, on='product_id', how='left')

# Bước 3.3: Nối thông tin Khách hàng (Customers)
# Để lấy customer_unique_id (dùng cho Gom cụm) và customer_state (dùng cho Map)
df = pd.merge(df, customers, on='customer_id', how='left')

# Bước 3.4: Nối thông tin Đánh giá (Reviews)
df = pd.merge(df, reviews, on='order_id', how='left')

# 4. Làm sạch dữ liệu (Tiền xử lý)

In [9]:
# 4.1. Chuyển đổi định dạng ngày tháng (Object -> Datetime)
date_cols = ['order_purchase_timestamp', 'order_approved_at', 
             'order_delivered_carrier_date', 'order_delivered_customer_date', 
             'order_estimated_delivery_date']

for col in date_cols:
    df[col] = pd.to_datetime(df[col])

In [10]:
# 4.2. Xóa dữ liệu trùng lặp
df = df.drop_duplicates()

In [11]:
# 4.3. Xóa các dòng đơn hàng bị hủy hoặc chưa giao (vì không phân tích được Review)
df = df.dropna(subset=['order_delivered_customer_date', 'review_score'])

In [12]:
# 4.4. Xóa cột thừa (Optional - Giúp file nhẹ hơn)
cols_to_drop = ['product_name_lenght', 'product_description_lenght', 'product_photos_qty']
df = df.drop(columns=cols_to_drop, errors='ignore')

In [13]:
# 4.5. Xóa giá tiền âm (Lỗi hệ thống)
df = df[df['price'] >= 0]

In [14]:
# 5.6. Xóa cột thừa (Optional - Giúp file nhẹ hơn)
import pandas as pd

def kiem_tra_cot_thua(df, nguong_null=90):
    """
    Hàm kiểm tra nhanh các cột tiềm năng thừa.
    - nguong_null: Tỉ lệ % Null tối đa cho phép (mặc định 90%)
    """
    print(f"🔍 ĐANG QUÉT {len(df.columns)} CỘT...")
    
    # Tạo bảng thống kê
    thong_ke = pd.DataFrame({
        'Ty_le_Null_Percent': df.isnull().mean() * 100,
        'So_gia_tri_duy_nhat': df.nunique(),
        'Vi_du_gia_tri': [df[col].dropna().iloc[0] if len(df[col].dropna()) > 0 else 'Rỗng' for col in df.columns]
    })
    
    # LỌC CÁC CỘT THỪA
    # Điều kiện 1: Null quá nhiều (> ngưỡng)
    # Điều kiện 2: Chỉ có 1 giá trị duy nhất (Variance = 0)
    cot_thua = thong_ke[
        (thong_ke['Ty_le_Null_Percent'] > nguong_null) | 
        (thong_ke['So_gia_tri_duy_nhat'] <= 1)
    ]
    
    if len(cot_thua) > 0:
        print(f"⚠️ PHÁT HIỆN {len(cot_thua)} CỘT NÊN XÓA:")
        return cot_thua.sort_values(by='Ty_le_Null_Percent', ascending=False)
    else:
        print("✅ Dữ liệu rất sạch! Không có cột nào vô nghĩa.")
        return None

# --- CÁCH SỬ DỤNG ---
# Giả sử bảng dữ liệu của bạn tên là 'df'
# Chạy hàm này để xem kết quả
df_cot_thua = kiem_tra_cot_thua(df)
df_cot_thua

🔍 ĐANG QUÉT 29 CỘT...
✅ Dữ liệu rất sạch! Không có cột nào vô nghĩa.


# 5. Tạo biến

In [15]:
# Tính thời gian giao hàng thực tế (Actual Delivery Days)
df['delivery_days'] = (df['order_delivered_customer_date'] - df['order_purchase_timestamp']).dt.days

In [16]:
# Tạo nhãn Giao trễ (Is Late)
df['is_late'] = np.where(df['order_delivered_customer_date'] > df['order_estimated_delivery_date'], 1, 0)

In [17]:
# Tạo nhãn Review Tốt/Xấu (Binary Classification)
# 1: Tốt (4-5 sao), 0: Xấu (1-3 sao)
# Xử lý Null Review thành -1 để không lỗi
df['review_score'] = df['review_score'].fillna(-1)
df['review_binary'] = np.where(df['review_score'] >= 4, 1, 0)

# 6. Kiểm tra & lưu file

In [18]:
# KẾT THÚC & XUẤT FILE
# ==============================================================================
# Lọc bỏ các dòng rác (không có sản phẩm)
df_final = df.dropna(subset=['product_id'])

# Lưu ra file Master để gửi cho nhóm
df.to_csv('master_data.csv', index=False)
print("XONG! File 'master_data.csv' đã được tạo thành công.")

XONG! File 'master_data.csv' đã được tạo thành công.
